In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import lightgbm as lgb
import shap
import matplotlib.pyplot as plt

In [3]:
data_path = 'D:\\jupyter files\\data_waiting_time_project\\preprocess_data\\'

In [4]:
data_part1 = pd.read_csv(data_path + 'data_part1.csv', index_col = 0)
data_part2 = pd.read_csv(data_path + 'data_part2.csv', index_col = 0)

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [5]:
data_part2_grouped = data_part2.groupby(['PatientSerNum', 'date'])
key_list = list(data_part2_grouped.groups.keys())

In [13]:

sample = data_part2_grouped.get_group(key_list[0])

new_treat = pd.DataFrame({
    'PatientSerNum': sample.PatientSerNum.tolist()[0],
    'FractionNumber': sample.FractionNumber.tolist()[0],
    'date': sample.date.tolist()[0],

    'UserName': [list(set(sample.UserName.tolist()))], 
    'RadiationSerNum': [list(set(sample.RadiationSerNum.tolist()))], 
    'RadiationId': [list(set(sample.RadiationId.tolist()))], 
    'ResourceSerNum': [list(set(sample.ResourceSerNum.tolist()))], 
    'CourseId': [list(set(sample.CourseId.tolist()))],

    'ImagesTaken_total': sum(sample.ImagesTaken.tolist()),
    'MU_total': sum(sample.MU.tolist()),
    'MUCoeff_total': sum(sample.MUCoeff.tolist()),
    'TreatmentTime_total': sum(sample.TreatmentTime.tolist()),
})

In [14]:
sample

,RadiationHstryAriaSer,TreatmentStartTime,TreatmentEndTime,FractionNumber,ImagesTaken,UserName,RadiationSerNum,RadiationId,ResourceSerNum,MU,MUCoeff,TreatmentTime,PatientSerNum,CourseId,date,Treatment_duration
0,3679490,2018-06-14 14:53:00,2018-06-14 14:53:00,1,0,rislam,632257,1.1 ANT,177,432.523,109.293,0.002833,83,C1,2018--06--14,0
1,3679491,2018-06-14 14:53:00,2018-06-14 14:53:00,1,0,rislam,632257,1.1 ANT,177,432.523,109.293,0.002833,83,C1,2018--06--14,0
3,3679492,2018-06-14 14:54:00,2018-06-14 14:54:00,1,0,rislam,632258,1.2 POST,177,434.852,109.881,0.002833,83,C1,2018--06--14,0
4,3679493,2018-06-14 14:55:00,2018-06-14 14:56:00,1,0,rislam,632258,1.2 POST,177,434.852,109.881,0.724500,83,C1,2018--06--14,60
2,3679494,2018-06-14 14:57:00,2018-06-14 14:57:00,1,0,rislam,632257,1.1 ANT,177,432.523,109.293,0.720617,83,C1,2018--06--14,0


In [15]:
new_treat

,PatientSerNum,FractionNumber,date,UserName,RadiationSerNum,RadiationId,ResourceSerNum,CourseId,ImagesTaken_total,MU_total,MUCoeff_total,TreatmentTime_total
0,83,1,2018--06--14,[rislam],"[632257, 632258]","[1.2 POST, 1.1 ANT]",[177],[C1],0,2167.273,547.641,1.453617


In [4]:
for col in data_part1.columns:
    print(col)
    break

dxt_AliasName


In [6]:
data_part1['dxt_AliasName'].unique()

array(['Breast', 'METS', 'Resp', 'Sarcoma', 'CNS', 'Prostate',
       'GU except prostate', 'Haeme', 'Skin', 'HN', nan, 'Gyne', 'LGI',
       'UGI', 'Eye', 'Other', 'Unknown Prim', 'HPB', 'ENT'], dtype=object)

In [26]:
print(f'The shape of data_part1 is {data_part1.shape}')
print(f'The shape of data_part2 is {data_part2.shape}')

DATA = pd.merge(data_part1, data_part2, on = ['PatientSerNum', 'date'], how = 'inner')
DATA.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
print(f'The shape of data is {DATA.shape}')

The shape of data_part1 is (1047176, 21)
The shape of data_part2 is (507863, 16)
The shape of data is (1788434, 35)


In [33]:
# data_grouped = data_.groupby('PatientSerNum')
# sample = data_grouped.get_group(6693)

In [30]:
# for col in data.columns:
#     print(f'\n{col}')
#     print(data[col].dtypes)
#     print(len(data[col].unique()))

In [40]:
# ['Sex', 'AliasSerNum',
#  'ScheduledStartTime', 'ScheduledEndTime',
# 'ActualStartDate', 'ActualEndDate', 'CourseSerNum',
# 'PlanSerNum', 'RadiationHstryAriaSer', 'TreatmentStartTime', 'TreatmentEndTime',
# 'RadiationSerNum', 'ResourceSerNum', ],

feature_num = ['Scheduled_duration', 'Actual_duration',
               'age', 'TreatmentTime', 'ImagesTaken',
               'MU', 'MUCoeff']

feature_cate = ['dxt_AliasName', 'Sex', 'AliasSerNum', 
                'month', 'date', 'week', 'hour', 'DoctorSerNum', 
                'TreatmentOrientation', 'FractionNumber',
                'UserName', 'RadiationId', 'CourseId']

In [62]:
print(data.shape)
data_ = DATA[feature_num + feature_cate]
data_.drop_duplicates(inplace = True)
print(data_.shape)

data_cate_display = data_[feature_cate]
data_cate_display.fillna('NaN', inplace = True)
data_cate_display = data_cate_display.astype(str)

data_num = data_[feature_num]
data_num.fillna(0, inplace = True)

(1788434, 35)



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(608291, 20)



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [83]:
# 数值变量暂时不做处理

# data_num = log1p(data_num)

# # y = np.log1p(data_num.age.tolist())
# y = data_num.age.tolist()
# y.sort()
# x = data_num.age.index
# plt.plot(x, y)
# plt.show()


# OrdinalEncoder 处理非数值变量
print('='*20)
print(f'\nEncoding features')
feature_encoder = preprocessing.OrdinalEncoder()
feature_encoder.fit(data_cate_display.values)
data_cate = feature_encoder.transform(data_cate_display.values)

# array 2 dataframe
data_cate = pd.DataFrame(data_cate, columns=data_cate_display.columns)
# 还需要换成category 格式，这样lightgbm 会按照类别变量的方式进行处理
data_cate = data_cate.astype('category')


# 拼接数据
print('='*20)
print('Concat data_num and data_cate')
data_num.reset_index(drop=True, inplace=True)
data_cate_display.reset_index(drop=True, inplace=True)
data_display = pd.concat([data_num, data_cate_display], axis=1)

data_num.reset_index(drop=True, inplace=True)
data_cate.reset_index(drop=True, inplace=True)
data = pd.concat([data_num, data_cate], axis=1)


Encoding features
Concat data_num and data_cate


In [88]:
data.columns

Index(['Scheduled_duration', 'Actual_duration', 'age', 'TreatmentTime',
       'ImagesTaken', 'MU', 'MUCoeff', 'dxt_AliasName', 'Sex', 'AliasSerNum',
       'month', 'date', 'week', 'hour', 'DoctorSerNum', 'TreatmentOrientation',
       'FractionNumber', 'UserName', 'RadiationId', 'CourseId'],
      dtype='object')

In [89]:
print('='*20)
print('Get train set and test set')
data = data.sample(frac = 1, random_state = 1)
data.reset_index(drop = True, inplace = True)

label = data[['Actual_duration']]
data.drop(['Actual_duration'], axis = 1, inplace = True)

train_x = data.iloc[: int(data.shape[0] * 0.9)]
train_y = label.iloc[: int(data.shape[0] * 0.9)]
print(f'\nThe number of train set is {train_x.shape[0]}')

test_x = data.iloc[int(data.shape[0] * 0.9) :]
test_y = label.iloc[int(data.shape[0] * 0.9): ]
print(f'The number of test set is {test_x.shape[0]}')

The number of train set is 547461
The number of test set is 60830


==============================================================================

In [107]:
# 创建模型，训练模型
gbm = lgb.LGBMRegressor(objective='mae', num_leaves=31, learning_rate=0.05,
                        n_estimators = 20000)
gbm.fit(
    train_x[: int(train_x.shape[0] * 0.9)], train_y[: int(train_y.shape[0] * 0.9)],
    eval_set=[(train_x[int(train_x.shape[0] * 0.1):], train_y[int(train_y.shape[0] * 0.1):])],
    eval_metric='mae',
    early_stopping_rounds=100,
    verbose=200
)
# 

Training until validation scores don't improve for 100 rounds
[200]	valid_0's l1: 6.13549
[400]	valid_0's l1: 5.75338
[600]	valid_0's l1: 5.58205
[800]	valid_0's l1: 5.46946
[1000]	valid_0's l1: 5.38439
[1200]	valid_0's l1: 5.31779
[1400]	valid_0's l1: 5.26572
[1600]	valid_0's l1: 5.21467
[1800]	valid_0's l1: 5.16847
[2000]	valid_0's l1: 5.11471
[2200]	valid_0's l1: 5.06291
[2400]	valid_0's l1: 5.01881
[2600]	valid_0's l1: 4.97059
[2800]	valid_0's l1: 4.93346
[3000]	valid_0's l1: 4.89394
[3200]	valid_0's l1: 4.84619
[3400]	valid_0's l1: 4.81235
[3600]	valid_0's l1: 4.7782
[3800]	valid_0's l1: 4.74401
[4000]	valid_0's l1: 4.71212
[4200]	valid_0's l1: 4.68764
[4400]	valid_0's l1: 4.66205
[4600]	valid_0's l1: 4.63597
[4800]	valid_0's l1: 4.60837
[5000]	valid_0's l1: 4.58145
[5200]	valid_0's l1: 4.55601
[5400]	valid_0's l1: 4.53426
[5600]	valid_0's l1: 4.51051
[5800]	valid_0's l1: 4.48688
[6000]	valid_0's l1: 4.4678
[6200]	valid_0's l1: 4.43947
[6400]	valid_0's l1: 4.4044
[6600]	valid_0's 

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=20000, n_jobs=-1, num_leaves=31, objective='mae',
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [94]:
pred_y = gbm.predict(test_x, num_iteration = gbm.best_iteration_)

In [95]:
pred_y

array([23.94709789, 34.38830407, 33.42804547, ..., 16.73550915,
       14.95060843, 29.2294304 ])

In [97]:
np.sqrt(2)

1.4142135623730951

In [102]:
pred_y

array([23.94709789, 34.38830407, 33.42804547, ..., 16.73550915,
       14.95060843, 29.2294304 ])

In [104]:
test_y.Actual_duration.tolist()

[16.0,
 34.0,
 36.0,
 11.0,
 26.0,
 18.0,
 10.0,
 10.0,
 17.0,
 9.0,
 11.0,
 15.0,
 24.0,
 21.0,
 13.0,
 16.0,
 18.0,
 16.0,
 14.0,
 22.0,
 15.0,
 35.0,
 8.0,
 16.0,
 36.0,
 14.0,
 43.0,
 27.0,
 12.0,
 14.0,
 14.0,
 11.0,
 18.0,
 11.0,
 13.0,
 17.0,
 17.0,
 62.0,
 25.0,
 16.0,
 18.0,
 14.0,
 16.0,
 19.0,
 27.0,
 68.0,
 16.0,
 41.0,
 67.0,
 46.0,
 27.0,
 40.0,
 12.0,
 47.0,
 14.0,
 18.0,
 11.0,
 11.0,
 12.0,
 20.0,
 39.0,
 14.0,
 16.0,
 16.0,
 11.0,
 10.0,
 28.0,
 29.0,
 21.0,
 18.0,
 27.0,
 28.0,
 27.0,
 23.0,
 32.0,
 43.0,
 14.0,
 24.0,
 9.0,
 9.0,
 19.0,
 17.0,
 21.0,
 15.0,
 21.0,
 21.0,
 28.0,
 12.0,
 33.0,
 68.0,
 43.0,
 18.0,
 12.0,
 12.0,
 37.0,
 116.0,
 16.0,
 23.0,
 16.0,
 23.0,
 18.0,
 34.0,
 16.0,
 10.0,
 20.0,
 12.0,
 13.0,
 10.0,
 21.0,
 14.0,
 13.0,
 15.0,
 20.0,
 17.0,
 13.0,
 24.0,
 11.0,
 10.0,
 20.0,
 11.0,
 29.0,
 14.0,
 7.0,
 55.0,
 13.0,
 14.0,
 17.0,
 10.0,
 26.0,
 30.0,
 27.0,
 20.0,
 17.0,
 41.0,
 22.0,
 13.0,
 32.0,
 25.0,
 21.0,
 21.0,
 21.0,
 19.0,
 24.0,
 48.0,
 12.0,
 18.0,
 22.0,
 13.0,
 15.0,
 12.0,
 13.0,
 29.0,
 16.0,
 9.0,
 27.0,
 23.0,
 13.0,
 12.0,
 29.0,
 12.0,
 42.0,
 12.0,
 6.0,
 17.0,
 38.0,
 16.0,
 28.0,
 19.0,
 10.0,
 18.0,
 12.0,
 35.0,
 16.0,
 16.0,
 25.0,
 7.0,
 12.0,
 16.0,
 19.0,
 22.0,
 33.0,
 15.0,
 8.0,
 34.0,
 27.0,
 15.0,
 28.0,
 25.0,
 59.0,
 17.0,
 17.0,
 29.0,
 26.0,
 26.0,
 10.0,
 15.0,
 46.0,
 21.0,
 37.0,
 30.0,
 14.0,
 17.0,
 26.0,
 10.0,
 15.0,
 22.0,
 22.0,
 16.0,
 15.0,
 15.0,
 18.0,
 18.0,
 15.0,
 18.0,
 18.0,
 8.0,
 16.0,
 12.0,
 42.0,
 18.0,
 10.0,
 31.0,
 14.0,
 22.0,
 58.0,
 11.0,
 22.0,
 18.0,
 13.0,
 32.0,
 24.0,
 30.0,
 11.0,
 10.0,
 37.0,
 15.0,
 24.0,
 17.0,
 18.0,
 29.0,
 10.0,
 16.0,
 17.0,
 26.0,
 14.0,
 10.0,
 21.0,
 49.0,
 45.0,
 22.0,
 18.0,
 12.0,
 58.0,
 18.0,
 7.0,
 20.0,
 10.0,
 29.0,
 10.0,
 21.0,
 17.0,
 20.0,
 9.0,
 14.0,
 26.0,
 47.0,
 45.0,
 6.0,
 20.0,
 8.0,
 30.0,
 24.0,
 17.0,
 17.0,
 20.0,
 19.0,
 50.0,
 8.0,
 20.0,
 13.0,
 20.0,
 22.0,
 20.0,
 28.0,
 13.0,
 34.0,
 11.0,
 16.0,
 18.0,
 20.0,
 68.0,
 9.0,
 28.0,
 19.0,
 9.0,
 21.0,
 19.0,
 25.0,
 11.0,
 22.0,
 15.0,
 17.0,
 22.0,
 14.0,
 19.0,
 15.0,
 15.0,
 37.0,
 20.0,
 12.0,
 53.0,
 75.0,
 14.0,
 31.0,
 14.0,
 22.0,
 20.0,
 29.0,
 10.0,
 18.0,
 18.0,
 15.0,
 34.0,
 19.0,
 34.0,
 23.0,
 17.0,
 26.0,
 17.0,
 10.0,
 23.0,
 11.0,
 14.0,
 30.0,
 8.0,
 20.0,
 4.0,
 19.0,
 6.0,
 11.0,
 21.0,
 100.0,
 23.0,
 22.0,
 16.0,
 23.0,
 16.0,
 11.0,
 11.0,
 79.0,
 23.0,
 17.0,
 24.0,
 14.0,
 72.0,
 5.0,
 14.0,
 31.0,
 12.0,
 27.0,
 18.0,
 18.0,
 93.0,
 13.0,
 36.0,
 11.0,
 14.0,
 11.0,
 9.0,
 13.0,
 10.0,
 13.0,
 37.0,
 93.0,
 20.0,
 24.0,
 47.0,
 32.0,
 18.0,
 18.0,
 15.0,
 56.0,
 16.0,
 11.0,
 14.0,
 29.0,
 17.0,
 16.0,
 4.0,
 21.0,
 27.0,
 8.0,
 22.0,
 8.0,
 15.0,
 10.0,
 14.0,
 32.0,
 15.0,
 25.0,
 15.0,
 16.0,
 9.0,
 24.0,
 36.0,
 32.0,
 18.0,
 15.0,
 9.0,
 11.0,
 14.0,
 12.0,
 13.0,
 30.0,
 27.0,
 14.0,
 42.0,
 26.0,
 25.0,
 33.0,
 29.0,
 15.0,
 12.0,
 12.0,
 40.0,
 24.0,
 15.0,
 11.0,
 15.0,
 19.0,
 13.0,
 26.0,
 12.0,
 20.0,
 24.0,
 11.0,
 16.0,
 73.0,
 10.0,
 25.0,
 18.0,
 16.0,
 15.0,
 33.0,
 12.0,
 19.0,
 12.0,
 28.0,
 23.0,
 9.0,
 54.0,
 19.0,
 22.0,
 13.0,
 13.0,
 18.0,
 10.0,
 22.0,
 12.0,
 52.0,
 19.0,
 13.0,
 18.0,
 11.0,
 36.0,
 34.0,
 31.0,
 23.0,
 28.0,
 16.0,
 20.0,
 13.0,
 56.0,
 109.0,
 35.0,
 40.0,
 12.0,
 15.0,
 14.0,
 12.0,
 14.0,
 28.0,
 10.0,
 16.0,
 12.0,
 7.0,
 13.0,
 19.0,
 11.0,
 17.0,
 18.0,
 34.0,
 23.0,
 29.0,
 25.0,
 45.0,
 6.0,
 83.0,
 30.0,
 31.0,
 13.0,
 16.0,
 17.0,
 22.0,
 14.0,
 13.0,
 33.0,
 18.0,
 30.0,
 13.0,
 10.0,
 22.0,
 15.0,
 16.0,
 12.0,
 11.0,
 21.0,
 14.0,
 16.0,
 33.0,
 8.0,
 14.0,
 15.0,
 12.0,
 7.0,
 28.0,
 17.0,
 21.0,
 20.0,
 9.0,
 13.0,
 15.0,
 23.0,
 16.0,
 32.0,
 14.0,
 10.0,
 9.0,
 42.0,
 15.0,
 10.0,
 24.0,
 19.0,
 14.0,
 25.0,
 13.0,
 16.0,
 17.0,
 17.0,
 21.0,
 51.0,
 9.0,
 12.0,
 27.0,
 14.0,
 14.0,
 19.0,
 14.0,
 24.0,
 16.0,
 26.0,
 14.0,
 20.0,
 9.0,
 15.0,
 44.0,
 25.0,
 15.0,
 13.0,
 22.0,
 15.0,
 19.0,
 16.0,
 17.0,
 9.0,
 20.0,
 7

In [105]:
sum((pred_y - np.array(test_y.Actual_duration.tolist()))**2)

2545942.8823967017

#### MSE 效果不好

In [7]:
def RMSE(pred_y, test_y):
    sq = sum((pred_y - np.array(test_y.Actual_duration.tolist()))**2) / len(pred_y)
    rmse = np.sqrt(sq)
    return rmse

In [106]:
np.sqrt(MSE(pred_y, test_y))

6.469421087895374

#### MAE

In [110]:
def MAE(pred_y, test_y):
    mae = sum(np.abs((pred_y - np.array(test_y.Actual_duration.tolist())))) / len(pred_y)
    return mae

In [111]:
MAE(pred_y, test_y)

1.8946490507418348